In [27]:
import pandas_datareader.data as web
import pandas as pd
import datetime
from decimal import Decimal
from dateutil import relativedelta
from backtesting import Backtest, Strategy
from backtesting.lib import crossover, resample_apply
import importlib
import plotly.express as px

#Change me!
start = datetime.datetime.today() + relativedelta.relativedelta(years=-5)
split_unit = "6M"
modulename = "BacktestingStrategies.Strategy_BBDandRSI"
classname = "btst.EntryRSI50andExitBBWithATRStopLoss"
end = datetime.date.today()

data = web.DataReader('1357.T', 'yahoo', start, end)  #NEXT FUNDS 日経平均ダブルインバース・インデックス連動型上場投信
data = data.astype("double")

initial = 1000_000
fname = f"html/plot-"

In [28]:
#日付データを検証期間単位ごとにSplit
#https://kakakakakku.hatenablog.com/entry/2021/05/24/002542
grouped_data = data.groupby(pd.Grouper(freq = split_unit))
dfs = [group for _, group in grouped_data]

In [29]:
#検証期間単位ごとにバックテスト＆結果を蓄積
#親ディレクトリ経由で呼び出し   # https://qiita.com/yokohama4580/items/466a483ae022d264c8ee
import os
import sys
sys.path.append(os.pardir)

btst = importlib.import_module(modulename)
# from BacktestingStrategies import Strategy_RsiOscillator as btst
importlib.reload(btst)
import warnings
warnings.simplefilter('ignore')

returns = []

for monthly_data in dfs:
    # バックテストを設定
    bt = Backtest(
        monthly_data, # チャートデータ
        eval(classname), # 売買戦略
        cash=initial, # 最初の所持金
        commission=0.00495, # 取引手数料
        margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
        trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
        exclusive_orders=True #自動でポジションをクローズ(オープン)
    )

    output = bt.run() # バックテスト実行
    returns.append(output)

In [30]:
import statistics

df = pd.DataFrame(returns)

#最低と最大の期間のBacktestingを表示＆実行
lines = "１期間単位: " + split_unit + "\n"
lines += "テスト期間: " + str(len(returns)) + "\n"
res = round(statistics.mean(df["Return [%]"]), 4)

#リターンが最小のものの期間を取得
min_return = df[df["# Trades"] != 0]["Return [%]"].min()
min_period_start = pd.to_datetime(df[df["Return [%]"] == min_return]["Start"]).values[0]
min_period_end = pd.to_datetime(df[df["Return [%]"] == min_return]["End"]).values[0]
df_min = data[min_period_start : min_period_end]

#リターンが2番目、3番目に小さいものの期間を取得
min_return_3 = df[df["# Trades"] != 0]["Return [%]"].nsmallest(3)
min2_period_start = pd.to_datetime(df[df["Return [%]"] == min_return_3.values[1]]["Start"]).values[0]
min2_period_end = pd.to_datetime(df[df["Return [%]"] == min_return_3.values[1]]["End"]).values[0]
min3_period_start = pd.to_datetime(df[df["Return [%]"] == min_return_3.values[2]]["Start"]).values[0]
min3_period_end = pd.to_datetime(df[df["Return [%]"] == min_return_3.values[2]]["End"]).values[0]
df_min2 = data[min2_period_start : min2_period_end]
df_min3 = data[min3_period_start : min3_period_end]

#リターンが最大のものの期間を取得
max_return = df[df["# Trades"] != 0]["Return [%]"].max()
max_period_start = pd.to_datetime(df[df["Return [%]"] == max_return]["Start"]).values[0]
max_period_end = pd.to_datetime(df[df["Return [%]"] == max_return]["End"]).values[0]
df_max = data[max_period_start : max_period_end]

lines += "平均Return [%]: " + str(res) + "\n"
lines += "最大Return [%]: " + str(max_return) + "\n"
lines += "最大Return期間: " + str(max_period_start) + " - " + str(max_period_end) + "\n"
lines += "最低Return [%]: " + str(min_return) + "\n"
lines += "最低Return期間: " + str(min_period_start) + " - " + str(min_period_end) 

print(lines)



１期間単位: 6M
テスト期間: 11
平均Return [%]: -1.6484
最大Return [%]: 19.410385034999997
最大Return期間: 2017-10-02T00:00:00.000000000 - 2018-03-30T00:00:00.000000000
最低Return [%]: -16.639641095000005
最低Return期間: 2018-10-01T00:00:00.000000000 - 2019-03-29T00:00:00.000000000


In [31]:
#取引ない月込み
fig = px.box(df["Return [%]"], points = "all")
fig.update_layout(
    xaxis_title = "Strategy",
    yaxis_title = "Return [%]"
)
fig.show()

In [32]:
#最低と最高の期間のバックテストを表示

print("----------WORST----------")
bt = Backtest(
    df_min, # チャートデータ
    eval(classname), # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True, #自動でポジションをクローズ(オープン)
)
output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + "_min.html") # 実行結果（グラフ）

print("----------MIN2----------")
bt = Backtest(
    df_min2, # チャートデータ
    eval(classname), # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True, #自動でポジションをクローズ(オープン)
)
output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + "_min2.html") # 実行結果（グラフ）

print("----------MIN3----------")
bt = Backtest(
    df_min3, # チャートデータ
    eval(classname), # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True, #自動でポジションをクローズ(オープン)
)
output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + "_min3.html") # 実行結果（グラフ）

print("----------BEST----------")
bt = Backtest(
    df_max, # チャートデータ
    eval(classname), # 売買戦略
    cash=initial, # 最初の所持金
    commission=0.00495, # 取引手数料
    margin=1.0, # レバレッジ倍率の逆数（0.5で2倍レバレッジ）
    trade_on_close=False, # True：現在の終値で取引，False：次の時間の始値で取引
    exclusive_orders=True, #自動でポジションをクローズ(オープン)
)
output = bt.run() # バックテスト実行
print(output) # 実行結果(データ)
display(output._trades)
bt.plot(filename = fname + str(output._strategy) + "_max.html") # 実行結果（グラフ）

----------WORST----------
Start                     2018-10-01 00:00:00
End                       2019-03-29 00:00:00
Duration                    179 days 00:00:00
Exposure Time [%]                   33.064516
Equity Final [$]                 833603.58905
Equity Peak [$]                 1024075.54215
Return [%]                         -16.639641
Buy & Hold Return [%]               17.227723
Return (Ann.) [%]                  -30.917275
Volatility (Ann.) [%]               13.305847
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -19.013632
Avg. Drawdown [%]                   -8.080481
Max. Drawdown Duration      109 days 00:00:00
Avg. Drawdown Duration       41 days 00:00:00
# Trades                                    7
Win Rate [%]                        14.285714
Best Trade [%]                       3.340921
Worst Trade [%]                     -7.196697
Avg. Tra

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-871,43,44,1147.29265,1163.0,-13681.10185,-0.013691,2018-11-29,2018-11-30,1 days
1,804,47,51,1226.03900,1267.0,32932.64400,0.033409,2018-12-05,2018-12-11,6 days
2,-840,54,55,1212.96595,1249.0,-30268.60200,-0.029707,2018-12-14,2018-12-17,3 days
3,-758,73,74,1303.51550,1314.0,-7947.25100,-0.008043,2019-01-16,2019-01-17,1 days
4,738,75,93,1327.53895,1232.0,-70507.74510,-0.071967,2019-01-18,2019-02-14,27 days
5,736,110,116,1237.09345,1171.0,-48644.77920,-0.053426,2019-03-11,2019-03-19,8 days
6,707,120,123,1217.99940,1178.0,-28279.57580,-0.032840,2019-03-26,2019-03-29,3 days


----------MIN2----------
Start                     2022-04-01 00:00:00
End                       2022-09-20 00:00:00
Duration                    172 days 00:00:00
Exposure Time [%]                   47.413793
Equity Final [$]                 856632.48295
Equity Peak [$]                  1018505.5272
Return [%]                         -14.336752
Buy & Hold Return [%]               -6.169666
Return (Ann.) [%]                   -28.55005
Volatility (Ann.) [%]               17.581378
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -20.074613
Avg. Drawdown [%]                  -20.074613
Max. Drawdown Duration      123 days 00:00:00
Avg. Drawdown Duration      123 days 00:00:00
# Trades                                    5
Win Rate [%]                             40.0
Best Trade [%]                       1.850609
Worst Trade [%]                     -10.08364
Avg. Trad

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-2348,30,31,425.88140,418.0,18505.52720,0.018506,2022-05-19,2022-05-20,1 days
1,-2552,32,33,399.01505,401.0,-5065.59240,-0.004975,2022-05-23,2022-05-24,1 days
2,2465,34,42,411.02455,384.0,-66615.51575,-0.065749,2022-05-25,2022-06-06,12 days
3,2270,48,74,417.05425,375.0,-95463.14750,-0.100836,2022-06-14,2022-07-21,37 days
4,2366,101,115,359.77210,362.0,5271.21140,0.006193,2022-08-30,2022-09-20,21 days


----------MIN3----------
Start                     2020-10-02 00:00:00
End                       2021-03-31 00:00:00
Duration                    180 days 00:00:00
Exposure Time [%]                   12.295082
Equity Final [$]                  909825.7297
Equity Peak [$]                     1000000.0
Return [%]                          -9.017427
Buy & Hold Return [%]              -42.243767
Return (Ann.) [%]                  -17.733094
Volatility (Ann.) [%]                9.913431
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -14.103484
Avg. Drawdown [%]                  -14.103484
Max. Drawdown Duration       61 days 00:00:00
Avg. Drawdown Duration       61 days 00:00:00
# Trades                                    4
Win Rate [%]                             25.0
Best Trade [%]                       5.934952
Worst Trade [%]                     -8.181773
Avg. Trad

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,2055,81,83,486.39580,455.0,-64518.3690,-0.064548,2021-02-01,2021-02-03,2 days
1,2115,103,109,442.17800,406.0,-76516.4700,-0.081818,2021-03-05,2021-03-15,10 days
2,2044,115,117,420.06910,445.0,50958.7596,0.059350,2021-03-23,2021-03-25,2 days
3,-2214,119,120,410.95565,411.0,-98.1909,-0.000108,2021-03-29,2021-03-30,1 days


----------BEST----------
Start                     2017-10-02 00:00:00
End                       2018-03-30 00:00:00
Duration                    179 days 00:00:00
Exposure Time [%]                   16.923077
Equity Final [$]                1194103.85035
Equity Peak [$]                 1194103.85035
Return [%]                          19.410385
Buy & Hold Return [%]              -16.502463
Return (Ann.) [%]                   41.040275
Volatility (Ann.) [%]               32.690174
Sharpe Ratio                         1.255432
Sortino Ratio                        4.359167
Calmar Ratio                         5.075445
Max. Drawdown [%]                   -8.086045
Avg. Drawdown [%]                   -4.936998
Max. Drawdown Duration       62 days 00:00:00
Avg. Drawdown Duration       28 days 00:00:00
# Trades                                    6
Win Rate [%]                             50.0
Best Trade [%]                       12.64534
Worst Trade [%]                        -5.401
Avg. Trad

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,743,48,50,1344.62310,1272.0,-53958.96330,-0.054010,2017-12-07,2017-12-11,4 days
1,767,87,91,1233.07365,1389.0,119595.51045,0.126453,2018-01-31,2018-02-06,6 days
2,-830,106,107,1282.61945,1303.0,-16915.85650,-0.015890,2018-02-27,2018-02-28,1 days
3,775,108,111,1352.66270,1398.0,35136.40750,0.033517,2018-03-01,2018-03-06,5 days
4,-801,116,117,1352.27295,1356.0,-2985.36705,-0.002756,2018-03-13,2018-03-14,1 days
5,776,120,125,1391.85575,1533.0,109527.93800,0.101407,2018-03-19,2018-03-26,7 days


Row(id='17289', ...)